In [1]:
import torch 
import numpy as np
from torch.autograd import Variable
from torch import nn
from torch.optim import SGD
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset


In [42]:
# making dataset for easy looping
class LowDimData(Dataset):
    def __init__(self, train, labels, transform=None):
        self.low_dim_embs = train
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.low_dim_embs)

    def __getitem__(self, idx):
        emb = self.low_dim_embs[idx]
        lab = self.labels[idx]
        
        sample = (emb, lab)
        if self.transform:
            sample = self.transform(sample)
        return sample

# for x in input_data:
#     print(x[1])
#     break


In [64]:
# # affinity regression:

# # RWA^T = Y
# # R:  low-level embeddings
# # W: weight matrix to learn
# # A: RNA/protein sequences to match
# # Y: known matches
## following class returns LHS 

class AfinityRegression(nn.Module):
    def __init__(self, emb_dim, rna_dim):
        super(AfinityRegression, self).__init__()
        # want to learn weights after applying to embedding layer (First do RW , then RW A^T)
        self.lin1 = nn.Linear(emb_dim, rna_dim)
    
    def init_weights(self):
        return None
    
    def forward(self, batch_size, embs, rna_samples):
        self.batch_size = batch_size
        x = self.lin1(embs)
        x = torch.matmul(x, rna_samples)

        return x



In [65]:
# dimensions and current dummy values
# samples = batch_size                              777
# embedding_dim = learned embedding dim (emb_dim)   10
# hidden1 = embedding_dim                           10
# hidden2 = dimension of rna samples (rna_dim)      2555
# rna_samples = # of RNA samples we're looking at   30


def training_loop(batch_size, num_epochs, model, optim, data_iter, rna_options):
    step = 0
    epoch = 0
    losses = []
    total_batches = int(len(data_iter) / batch_size)
    poss_matches = Variable(rna_options)
    while epoch <= num_epochs:
        for x in data_iter:
            learned_embs = Variable(x[0])
            known_matches = Variable(x[1])
            
            model.train()
            model.zero_grad()

            outs = test_model.forward(3, learned_embs, poss_matches)
            new_mat = known_matches - outs
            loss = torch.bmm(new_mat.view(new_mat.size()[0], 1, new_mat.size()[1]),
                               new_mat.view(new_mat.size()[0], new_mat.size()[1], 1)
                              )
            loss = torch.sqrt(loss)
            loss = torch.div(torch.sum(loss.view(-1)), batch_size)
            losses.append(loss[0].data.numpy()) 
            loss.backward()
            optim.step()


        epoch += 1
#         if epoch % 2 == 0:
        print( "Epoch:", (epoch), "Avg Loss:", np.mean(losses)/(total_batches*epoch) )
        
        step += 1




In [66]:
# dummy data
learned_embs = torch.randn((777,10))
poss_matches = torch.randn((2555, 30))
known_matches = torch.randn((777,30))

test_model = AfinityRegression(emb_dim=10, rna_dim=2555)
optimizer = SGD(test_model.parameters(), lr = 0.001)
test_model.init_weights()
# input_data = (learned_embs, known_matches)
batch_size = 5
num_epochs = 5
input_data = DataLoader(LowDimData(learned_embs, known_matches), batch_size=batch_size)
training_loop(batch_size, num_epochs, test_model, optimizer, input_data, poss_matches)


Epoch: 1 Avg Loss: 1.21311803018
Epoch: 2 Avg Loss: 0.362865263416
Epoch: 3 Avg Loss: 0.187756015408
Epoch: 4 Avg Loss: 0.12050919379
Epoch: 5 Avg Loss: 0.0866596098869
Epoch: 6 Avg Loss: 0.0668009224758
